# メモ

### なぜこんなものを作ろうと思ったか

1. 30年前に、Resolution Principleにもとづく証明系を用いたあるシステムの開発に関係していた。
1. そのときの言語はINTERLISP(-D)だった
1. 記号処理にLISPは最適だと思うようになった。
    - たとえばSmalltalkでも同じ処理を書くことはできるが、「記号」をtransparentに扱う(つまりLISPのatom)ことができないのは、直感的に違和感があった。他の関数型言語でも同じ。
1. Resolution を用いた証明系を作ろうかと思うことは何度かあったが、どの言語もこれはという印象がなかった。COMMONLISPでいくつかプログラムを作ったこともあるが、COMMONLISPの場合は、必要なものを自作しないとならない印象があって、日常に使うのはためらわれた。
1. ここ数年、Juliaで簡単なプログラムを作ってきた。その前の数年は、Rでプログラムを組んでいたが、それでResolutionを作るという気にはならなかった。
1. Juliaは気に入ったが、これでResolutionを作れるかというと、大変になるような気がしたので真面目に考えてはいなかった。
1. 昨年末に、次のような点に気づいたので、Juliaでproverを作ってみようと思い始めた。
    - Juliaでシンボルは:xyzのように書くことになるわけだが、これを xyz = :xyz とすれば、あたかもatomのように書けるということ。
    - formulaがある程度LISPのように簡単に書ける。
        - つまり[[:x,:y],[:+ :P :x],[:- :Q :x [:f :y]]] のような配列でformula(cnf)を表せる
        - この配列表現は書くのに手間がかかるけれど、parse("[x,y].[+P(x),-Q(x,f(y))]")とすると、文字列の中はJuliaの式として妥当であり、parse()の結果のExprはformulaとして使うことが難しくない。つまり、このルールで書いたformulaは意外と簡単に書ける。
            - ここではCNFしか想定していない。


### これまでのいきさつ

1. まず、proverを使う目的を考えた。
    - 単に証明を作るというのは漠然としているし、数学の定理の証明が非決定的であり、面倒臭いことはわかっていた。
    - 30年前から興味があったのはproverをどう作るかの前に、世界をどうやって論理式で表現できるかだった。
    - プログラムのいろいろな性質を論理式で表現するとき、計算自体を論理式で表現することはできるが、計算がやろうとしていること、最終的にはそのプログラムが解決する問題とプログラムの機能の間の関係を論理式で表現し、その関係を証明すること。この、何段階にもなる論理式の関係であるとか、人間の意図をどうやって論理式で表現するか、などに興味があった。
    - 数学の命題は論理式でうまく表現できる。では、日常にあるいろいろなできごとを表現するにはどうすればいいか。
1. LogicalWorldに作ったのは、その観点から作った0.5階論理のproverだった。
    - 世界の記述では、すべての事実はground unit clauseになる。
    - その事実に対するqueryが一階の論理式で表される。
    - 基本的には命題論理なので、非決定性はない。
    - 複数のquery式と、Factの間のmguをモデルと呼び、全queryを真にするモデルを求めたりするということをした。
    - しかし、表現力がたりないのか、うまく書けないこともあったので、一回述語のresolutionを作ることにした。
1. Resolution のプログラムの構造についてはかなり忘れていた。Logical Worldは基本的に命題であり、変数はqueryだけにあるから、変数間のmgu(inside含む)を考慮していない。
    - prover(仮)を作る過程で、いろいろ調べながら思い出しながら機能を充実させていった。
    - 30年前は、Chang&Leeの本くらいしかなく、そこに書かれていたプログラムがベースになっていたが、今回、スクラッチで作る上で、いくつか改良(と思っている)変更を加えている。
    - 前提として、Unit で input Resolutionになると考えている。
    - proofが進み、resolventがたくさんできてくるけれど、そのresolventのliteralはすべてinput clauseにその起源がある。
        - 同じclauseが複数回使われるようなproofでは、同じliteralが何度も出現するが、input clause以外のliteralは出現しない。
        - であれば、input clauseのすべてのliteralにidをつけ、resolventのliteralをそのidで参照することを考えられる。
        - 実装では、clauseについてはinput clauseのid(cid)、resolventのid(rid)をつけ、literalのidは、lidとしてLn_CmRpという名前をつける。Cmはcid, Rpはrid。
        - こうした場合、clauseを保持していても仕方がないので、literal単位にバラバラに管理し、必要なときclauseを再構築するということにした。
        - 変数の扱いは面倒だった。
            - lidに対してvarsを持つことも考えたが、変数はclauseを中心に複数のliteralに共通な変数があるので、cid/ridに対してvarsを持つようにした。
    - resolutionの前に、2つのclauseは共通の変数を持ってはならないので、変数のrenameということをする必要がある。
        - ChangLeeでは、resolutionの前に共通の変数を求めて、名前を付け替えていたと思う。
        - 効率がよくなると考えた方法は、resolventができたときに変数名をかえ、常にすべてのclauseの変数名が違うようにすること。
            - 2つのclauseの変数名に共通がないという条件をDistinct Variable Condition(DVC)と勝手によんで、そのほうほうを作った。
            - resolventの変数名をすべて違うようにすることで、resolutionの前処理としてのrenameは不要になり、renameの回数を考えると、この方法の方が効率が良い。
            - ではresolventの変数にどのような名前をつけるのか。以下、元の変数をVとする。
                - input clauseの変数には、V_C1 のような名前をつける
                - resolventの変数は、V_C1R2のような名前にする。このとき、V_C1R2が別のresolvent R8に出現する場合は、V_C1R8というようにridの部分を置換する。
                - というルールである。
        -
    - proofの各ステップはresolventを作るときunifyされる2つのリテラルで決まる。
        - literal pairは、それをunifyしてできるmguと同等である。
            - input/linear resolutionを想定しているので、proofとはこのliteral pairのシーケンスであり、mguのシーケンスである
        - proof全体の構造は、clauseの構造で決まる。
        - clauseによって決まる変数の同一性も重要。
    - proofを作って行く上で問題になるのは、不要なresolventをどう作らないかだと思う。
        - とりあえず考えた方法
            - 同じliteral pairが繰り返し出現する場合、それ以上そのパターンは不要とする。
                - 述語論理の場合、これは繰り返しになるので、繰り返した果てにある値になりそこではじめてあるliteralとunifyされるというような証明はありうるが、それは算術の世界では重要だが、世界の記述ではそれほどないと思うので、簡単にはじいた
            - resolventからunifyするliteralを選択するとき、左端からみつかるまで探している。これは明らかに不完全になるのでなんとかしなくてはならない。
            
1. 解決したい問題(再度)

### 論理を使うメリットは何か?
1. たとえ何かを論理式で書いたとしても、推論部分を自動証明でできるなら、proverを使わずに、programでも書ける。では自動証明を使うメリットはなにか?
1. プログラムでは得られないツールが考えられる。
    - Axiomの無矛盾性を証明できるならば、プログラムではできない。
        - templateにして、欠落があればそれはのぞいて良いなど
    - 入出力を指定せずに構造を定義する。
        - プログラムよりも広範囲の・・・モデルを記述できるメリット
1. 論理式でプログラムを書くのがPrologだった。そんなに詳しくないが、Prologの式をprogramと解釈せずに証明することはできたのだろうか。
    - proof=programであるとすれば、Prologではproofの構成を制御する操作(証明内容には関係しない)が使えるということだから、programの実行自体がproofということになり、上の問いに対する答えはできたということになる。
    - しかし、Prologのprogramは計算を頭においた論理式になるので、私の目的とは違っている。
    - 私は世界をそのまま書きたい。(「そのまま」が未定義なのでこれはただのスローガンでしかない)
    - あるいは、世界をどう書けばありのままになるのか?

### 機械学習と自動証明(動機)

機械学習(深層学習)のことは詳しくは知らないが、いくつかのシステムを見たところ、画像を与えると、その画像が属する、あるいは含む何かについて、タグとその確率(?)を判定してくれるらしい。画像以外のものはよくしらない。

この数字が確率であれば、それに基づいてベイズを使うのかどうか知らないが、推論をするという話もあるらしい。

一方で、自動証明あるいは機械証明というものがあって、ここではResolution Principleを使うのしか考えないが(SAT solverだったりSMTなどもその一分野だと思う)。
Resolutionは推論のルールで、これによって推論を積み重ねていく。この場合の推論は、上の確率ベースの推論とは違うものだろうと思う。

今は、この二つの分野を連携するという話をあまり聞かないけれど、うまく繋げることができるのではないかというのが、当面の目標目的?。

2. では、どうつなげるかという話になるのだが、今の段階ではそれほど明確ではない。ぼんやりとしたイメージとして、次のようなものを持っているが、よりよいものに変わるかもしれない。

    1. ML(機械学習)で、写真に含まれる概念を抽出する。
        1. これを児童証明のwffにする。たとえば
            ``` P(x, 0.9) ```
            で、'ある画像xがPである確率0.9'を示す。PはMLの出力のタグに相当する
            画像xは、複数の述語で書かれ得るので、Factとして
            ```
            P(x,0.9)
            Q(x,0.8)
            R(x,0.99)
            ```
            のようになる。
        2. MLによってFactの集合を得て、そのうえで何か自明でないことを証明する
        3. できれば、Fact上で成り立つ性質を自動的に発見する。

3. こういうことを実現するために、次のような仕組みが必要なのだろうと思っている。
    - いろいろな世界を記述する方法の洗練
    - 確率に基づく推論での自動証明
    - 時間に基づく推論での自動証明
    - 等号(=)を含む推論(SMTにつながる)
    - Factの抽象化、一般化
    - ...
    
4. 個別の分野は既存だが、問題設定が違うから、それなりにしなくてはならないことがあるだろうと思っている。


### 機械学習について

どこかに書いたはずだけど、みつからないので書き直す

1. 高次元(N次元)空間のある分布の特徴を抽出するのが機械学習だと思う。
1. 

1. 個々のドローンの特徴を決めるのは、初期値と夢ではないだろうか。
    1. 夢というのは、そのドローンがどこにむかって進むかの方針
    1. 基本的にその夢が破綻してもよい。
    1. 自然淘汰されて、生き残ったドローンの知識を使えばよいから。
1. 既存の知識にしばられるドローンもいてもよいし、しばられないものがいてもよい。
    1. メタなレベルでも自由
    1. わかっていないことを分かるようになったら成熟したといってもよいが
        1. そのような認識はどのような知識として得られるのか?
    1. 学習が進むと、行動が変わっていくのは当然
    

### 世界の書き方
何かを書きたい時、どうやって書くかは、昔から難しいと思っていた。
道具もできたことなので、いろいろ試してみることにする。


#### conjectureの意味(3.25) 

1. (3.25) conjectureの意味はなんなんだろうか

* (id1.wff) Axiomsを次のようにとったとき、conjは+E(x,x)であり、Axiomsが2から3のときはで常にこの関係が成り立っているということを証明でき、Aiomsに4を加えると、d,eについて成り立たないことが示せないかと期待したのだが、そうはいかない。

```
1 [x].[-E(x,x)]
2 [].[+E(a,a)]
3 [].[+E(b,b)]
4 [].[+E(d,e)]
```
このとき、aとbについては[]がでるが、d,eについては出ないので、この証明集合の全体をみると、{2,3}については1が成り立っていると判断できるが、何かひとつの証明でそれができているわけではない。

もしもそのような証明がしたかったら1のように変数で示すのではなく
```
-E(a,a)
-E(b,b)
-E(d,d)
-E(e,e)
```
と展開したconjectureを証明することになるはずだ。

その場合、Axiomに+E(a,a)などが存在していたら、この方法は1段階の証明でしか証明できないのでトリビアルになり、意義はない。

私は何がしたいのだろうか?

2. もしも、-E(x,x)となる定数が存在すれば、1は否定できる。とすると
```
1'. [x].[+E(x,x)]
```
    だと、何も証明できない。反証できない。
3. Axiomsに[x].[+E(x,x)]をいれておけばいいのか?　conjをどうするかがわからないが、まずdと同じものを探してみる。

```
1 [x].[-E(d,x)]
2 [].[+E(a,a)]
3 [].[+E(b,b)]
4 [].[+E(d,e)]
5 [x].[+E(x,x)]
```
    こうすると、1のxに代入されるrefutationでは、{x←d}と{d←e}がみつかるので、なんとなくうれしい。

4. この方向なら、1を$[x].[-E(x,x)]$とすれば等しいものがわかるのかと思ったが、それはEが等しいものを定義しているという意味ではないので却下。

1を$[x,y].[-E(x,y)]$とするとEを満たすすべてのpairを求めることになるのでこうすると

```
1 [x,y].[-E(x,y)]
2 [].[+E(a,a)]
3 [].[+E(b,b)]
4 [].[+E(d,e)]
5. [x].[+E(x,x)]
```
でx,yにすべての組み合わせの結果が代入される証明が得られる。
これ自体はトリビアルだが、何か発展できないか?

5. この方法は、DBで制約条件を増やしてそれを満たすデータを求めているのと同じ

```
1. -E(x),-D(x)
2. E(a)
3. E(b)
4. D(a)
5. D(c)
```
で、conjに着目すると{x←a}の証明が得られるのと同じ。

6. 次の操作には意味があるのではないか
    1. 得られたrefutaionから情報を取り出す操作
    2. 



### 半順序のようなもの

(a) $\forall x,z \exists y +H(x,y)\land +H(y,z)\to +H(x,z)$

eigen変数を除去すると
(a') $\forall x,z +H(x,y(x,z))\land +H(y(x,z),z)\to +H(x,z)$




これは間違っている
$\forall x,z,y +H(x,y)\land +H(y,z)\to +H(x,z)$




### Fact$(\Phi)$とHypo$(\Theta)$

$\Theta$が$\Phi$の法則だということはどういう意味か?
つまり、「Factが与えられて、そのFactの上で成り立つ法則を見つけたい」ので。

1. $\Phi$を操作して$\Theta$が求められるか? むりではないか

2. 法則であることを証明する手段はあるか? そもそも法則だというのはどういう定義になるのか

a) まず、$\Phi \vdash \Theta$であるということだろう。では証明は?

$\Phi\mid_{\phi}^*$を$\Phi^{'} \nvdash \phi$であるとする。ここで、$\Phi^{'}$は$\Phi$から$\Phi^{'} \nvdash \phi$となるまで要素をのぞいたものとする。
    唯一にはならないかもしれない。
    

#### 2つの方法の比較

1. Factに対して、ある仮説が成り立っているかどうかを判定するには二つの方法がある(他にもあるかしれない)
    1. 仮説Hに対して、その変数にFactのすべての定数を代入(すべての組み合わせ)したformulaをつくり、これの証明を行う。
        1. ひとつでも真にならなければ、その仮説はなりたたない
        1. しかし、すべての定数なのか? 特定の定数ではないのか? その変数に対応する型に属する定数だけではないのか。
            - それは、どような論理を使うかによるのだろう。
    1. 仮説Hの否定の証明をすべて求める(定数は有限個なので有限回で終わるか?? 関数があると無理だろう)
        1. 得られた証明から、仮説の変数に対して、Factの定数がすべての組み合わせについて代入されていることを確認する。
    1. 前者のほうが確実な気がする。

1. Factのすべてで仮説Hが成り立つことなど知りたいわけではないかもしれない。
    1. Factから、仮説Hを作る。Hが常に成り立つことは証明できないかもしれない。
    2. Hのもとで、Factから、ある予想が得られる。R(a,c)のようなもの。
    3. それを現実世界で確認して、+R(a,c)か-R(a,c)かをFactに追加する。
2. というのが、使えるのではないか
3. このとき、Hが常に成り立つことを言うよりも、役に立つ定理R(a,c)を見つけることのほうが重要だと思う。
4. 仮説Hを成り立たせるための未検証のpseudo Fact R(a,c)という考え方
    1. 仮説Hに足りないものであるとか・・



1. 全順序の性質などは、頭の中ですぐに証明できるが、それは対象世界にもともと全順序が定義されているから。
2. あるいは、対象世界を構成する手続きの中に、全順序を保証する性質があるのだろう。


未完

### 確率論理

1. $R(..., prob)$と書くことにする。

もとい

1. $R(x)/p$と書いてみる。つまり、xの存在する確率がpなのか、R(x)が真である確率がpなのか。


ある絵があったとき、そこに描かれているものが存在すると信じる理由はなにか?

そこには花が描かれているのかもしれないし、火が描かれているのかもしれない。というような状況を考える。

その絵の中に、あるものRが存在するように見える確率

花の確率は、そこに薔薇が描かれている確率、百合が描かれている確率、などすべての花についての確率の和になるだろうか。実際には、みかけの区別できないものもあるので、合計ではなさそう。

$\forall f \in Flower\, \Sigma_f Pr(f,x) \ge Pr(Flower, x)$

ただし、$Pr(A,x)$とは、xがAの画像である確率。

こういう法則をいろいろ考えていくのかな

$R(x,y,p)$で、xの中にyが出現している確率がpである。という表現もありそう。



学習について

何億もの試行をし

その結果にもとづいて conpetitionをし

優勢な試行を残して行く。

もったいないので、負けたほうのモデルも残すと混乱するのかも。

何億人もの子供に競争をさせて、両方の経験を集約していく。

Alpha GOハドウヤッテイタノダロウカ

N次元空間があって、その各点にドローンを置き、局所的な情報を学習したとする。
それらのドローンの知識をcompetitionで比較していき、優秀なものを残すという戦略
ある目的に対して、どの場所の知識が有益かとかはわからないが、この方法で適応した個体が残される。

生命の進化における適用はそのようなスキームではないか??

統計的に何億もの試行は不要である、ということになるのだろう。
自然は統計をせず、莫大な空間と時間を使って試行を行ってきた。

空間に歪みがあれば、場所によって必要な試行も数も変わるだろう。

いったい何を学習するのかということもある。

人間は、子供の頃の育ち方で、考え方が構成される。体験は同一ではあり得ない。

そのような子供を、競争させることで、より適応力の高い者を選択する。

そのためには、最終的な目的にふさわしい環境での競争が必要。

AI研究では、自然がこれまで行ってきた実験結果を単純に集約して、計算機械に投入している。

釣りか魚かでいうと、魚。

釣りを教えるとすると、
AI自体が進化するような仕組みがあれば、どんな問題にも対応できるという仮説に立つならば、もっと抽象的な方法があるだろう。


#### 確率と否定の関係?
1. R(x,1.0)は、xの起きる確率が1.0。R(x,0.0)はxが決して起きないということを意味する。
2. -R(x,1.0)の意味は何か? 
3. -R(x,0.0)の意味はなにか?何にしたいか?
4. 否定とはなにか?

#### 確率世界と確定世界の関係
5. $ R(a,1.0) \equiv R(a)$と考えることにしよう

#### 論理
1. aからdへのパスが二つあるとする。$a \to b, a \to c, b \to d, c \to d$
1. 確率は、$Pr(a,b)=0.5, Pr(a,c)=0.5, Pr(b,d)=1.0, Pr(c,d)=1.0$でこれ以外にパスはない。とする。
    これ以外にパスがないことをどう表すか?
1. このとき
    $R(a,b,0.5), R(a,c,0.5), R(b,d,1.0), R(c,d,1.0) \to R(a,d,1.0)$のようなことが成り立てばよい。
1. もっと簡単な場合は
    $R(a,b,1.0), R(b,c,1.0)\to R(a,c,1.0)$
    

#### 雑記

1. 型について
    - f(x::T)という関数fがあったとき、型Tは、その関数の中でxが使われているその使われ方のみを規定すればいい。(これはtemplateと同じかも。Dylanの本にはこういう話があったと思う)
    
    - f(x::T)の中でxに+と-のみ行われていたら、fにおいてはxの型Tは+と-のみ規定すればよい。
    - だとすると、fの中で必要な型情報は、T全体ではないので、fのxに関して必要な型をT(f,x)と書くならば
        ```
        T(f,x) <: T
        ```
      のように書かれるだろう。
    - 

参考資料
- The Resolution Calculus, Alexander Leitsch, Springer-Verlag Berlin Heidelberg New York, 1997
- Symbolic Logic and Mechanical Theorem Proving, Chin-Liang Chang and Richard Char-Tung Lee, Academic Press, 1973
- LOGIC: FORM AND FUNCTION The Mechanization of Deductive Reasoning, J.A.Robinson, Edinburgh University Pres, 1979
